## Data description
There are two data sources for this assignment. They are DataFrames in parquet format.

The first dataset captures the user’s playing history.

Location - /data/sample264

Fields: trackId, userId, timestamp, artistId

* trackId - id of the track
* userId - id of the user
* artistId - id of the artist
* timestamp - timestamp of the moment the user starts listening to a track

The second is the meta data for track or artist.

Location - /data/meta

Fields: type, Name, Artist, Id

* Type could be “track” or “artist”
* Name is the title of the track, if the type == “track” and the name of the musician or group, if the type == “artist”.
* Artist states for the creator of the track in case the type == “track” and for the name of the musician or group in case the type == “artist”.
* Id - id of the item

In [1]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [2]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

# Graph based Music Recommender. Task 1
Build the edges of the type “track-track”. To do it you will need to count the collaborative similarity between all the tracks: if a user has started listening to track B within 7 minutes after starting track A, then you should add 1 to the weight of the edge from vertex A to vertex B (initial weight is equal to 0).

Example:

<code>
userId artistId trackId timestamp
7        12        1          1534574189
7        13        4          1534574289 
5        12        1          1534574389 
5        13        4          1534594189 
6        12        1          1534574489 
6        13        4          1534574689
</code>

The track 1 is similar to the track 4 with the weight 2 (before normalization): the user 7 and the user 6 listened these 2 tracks together in the 7 minutes long window:

<code>
userId 7: 1534574289 - 1534574189 = 100 seconds = 1 min 40 seconds < 7 minutes
userId 6: 1534574689 - 1534574489 = 200 seconds = 3 min 20 seconds < 7 minutes
</code>

Note that the track 4 is similar to the track 1 with the same weight 2.

Tip: consider joining the graph to itself with the UserId and remove pairs with the same tracks.For each track choose top 50 tracks ordered by weight similar to it and normalize weights of its edges (divide the weight of each edge on a sum of weights of all edges). Use rank() to choose top 40 tracks as is done in the demo.

Sort the resulting Data Frame in the descending order by the column norm_weight, and then in the ascending order this time first by “id1”, then by “id2”. Take top 40 rows, select only the columns “id1”, “id2”, and print the columns “id1”, “id2” of the resulting dataframe.

Output example:

<code>
54719		767867
54719		767866
50787		327676
</code>

In [3]:
from pyspark.sql import Window
import pyspark.sql.functions as f

In [4]:
def normalize(df, key1, key2, field, n):    
    window = Window.partitionBy(key1).orderBy(f.col(field).desc())
    topsDF = df.withColumn("row_number", f.row_number().over(window)) \
        .filter(f.col("row_number") <= n) \
        .drop(f.col("row_number"))
    tmpDF = topsDF.groupBy(f.col(key1)).agg(f.col(key1), f.sum(f.col(field)).alias("sum_" + field))
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, f.col(field) / f.col("sum_" + field))
    return normalizedDF

In [ ]:
'''
w = Window.partitionBy('userId').orderBy('timestamp')

tracks = data \
    .select(
        'userId',
        'trackId',
        f.lag('timestamp').over(w).alias('timestamp_lag'),
        f.lag('trackId').over(w).alias('trackId_lag')) \
    .filter(f.col('timestamp') - f.col('timestamp_lag') < 7*60) \
    .select(f.col('trackId').alias('id1'), f.col('trackId_lag').alias('id2')) \
    .groupBy('id1', 'id2').agg(f.count("*").alias('cnt1')) \
    .cache()
    
reverted = tracks.select(
    f.col('id2').alias('id1'), 
    f.col('id1').alias('id2'), 
    f.col('cnt1').alias('cnt2'))

df = tracks.join(reverted, on=['id1','id2'], how='left') \
    .select(
        'id1', 
        'id2', 
        (f.col('cnt1') + f.when(f.isnull('cnt2'), 0).otherwise(f.col('cnt2'))).alias('count'))
''';

In [ ]:
def task1():
    A = data.select('userId', f.col('trackId').alias('track1'), f.col('timestamp').alias('ts1'))
    B = data.select('userId',f.col('trackId').alias('track2'), f.col('timestamp').alias('ts2'))

    AB = A.join(B, on=['userId']) \
        .filter((A['track1'] != B['track2']) & (f.abs(A['ts1']-B['ts2']) <= 7*60)) \
        .groupBy('track1', 'track2') \
        .agg(f.count("*").alias('count')) \
        .cache()

    df = normalize(AB, 'track1', 'track2', 'count', 1000) \
        .orderBy(f.desc('norm_count'), 'track1', 'track2') \
        .select('track1', 'track2') \
        .limit(40)

    for t1, t2 in df.collect():
        print("{}\t{}".format(t1,t2))

#task1()

798256	923706
798319	837992
798322	876562
798331	827364
798335	840741
798374	816874
798375	810685
798379	812055
798380	840113
798396	817687
798398	926302
798405	867217
798443	905923
798457	918918
798460	891840
798461	940379
798470	840814
798474	963162
798477	883244
798485	955521
798505	905671
798545	949238
798550	936295
798626	845438
798691	818279
798692	898823
798702	811440
798704	937570
798725	933147
798738	894170
798745	799665
798782	956938
798801	950802
798820	890393
798833	916319
798865	962662
798931	893574
798946	946408
799012	809997
799024	935246


# Graph based Music Recommender. Task 2
Build the edges of the type “user-track”. Take the amount of times the track was listened by the user as the weight of the edge from the user’s vertex to the track’s vertex.

Tip: group the dataframe by columns userId and trackId and use function “count” of DF API.

For each user take top-1000 and normalize them.

Sort the resulting Data Frame in descending order by the column norm_weight, and then in ascending order this time first by “id1”, then by “id2”. Take top 40 rows, select only the columns “id1”, “id2”, and print the columns “id1”, “id2” of the resulting dataframe.

The part of the result on the sample dataset:

<code>
...
195 946408
215 860111
235 897176
300 857973
321 915545
...
</code>


In [ ]:
def task2():
    X = data.select('userId', 'trackId').groupBy('userId', 'trackId').agg(f.count("*").alias('count')).cache()

    df = normalize(X, 'userId', 'trackId', 'count', 1000) \
        .orderBy(f.desc('norm_count'), 'userId', 'trackId') \
        .select('userId', 'trackId') \
        .limit(40)

    for t1, t2 in df.collect():
        print("{}\t{}".format(t1,t2))
        
#task2()

66	965774
116	867268
128	852564
131	880170
195	946408
215	860111
235	897176
300	857973
321	915545
328	943482
333	818202
346	864911
356	961308
428	943572
431	902497
445	831381
488	841340
542	815388
617	946395
649	901672
658	937522
662	881433
698	935934
708	952432
746	879259
747	879259
776	946408
784	806468
806	866581
811	948017
837	799685
901	871513
923	879322
934	940714
957	945183
989	878364
999	967768
1006	962774
1049	849484
1057	920458


# Graph based Music Recommender. Task 3
Build the edges of the type “user-artist”. Take the amount of times the user has listened to the artist’s tracks as the weight of the edge from the user’s vertex to the artist’s vertex.

Tip: group the dataframe by the columns userId and trackId and use the function “count” of DF API. For each user take top-100 artists and normalize weights.

Sort the resulting Data Frame in descending order by the column norm_weight, and then in ascending order this time first by “id1”, then by “id2”. Take top 40 rows, select only the columns “id1”, “id2”, and print the columns “id1”, “id2” of the resulting dataframe.

The part of the result on the sample dataset:

<code>
...
131 983068
195 997265
215 991696
235 990642
288 1000564
...
</code>

In [ ]:
def task3():
    X = data.select('userId', 'artistId').groupBy('userId', 'artistId').agg(f.count("*").alias('count')).cache()

    df = normalize(X, 'userId', 'artistId', 'count', 1000) \
        .orderBy(f.desc('norm_count'), 'userId', 'artistId') \
        .select('userId', 'artistId') \
        .limit(40)

    for t1, t2 in df.collect():
        print("{}\t{}".format(t1,t2))

#task3()

66	993426
116	974937
128	1003021
131	983068
195	997265
215	991696
235	990642
288	1000564
300	1003362
321	986172
328	967986
333	1000416
346	982037
356	974846
374	1003167
428	993161
431	969340
445	970387
488	970525
542	969751
612	987351
617	970240
649	973851
658	973232
662	975279
698	995788
708	968848
746	972032
747	972032
776	997265
784	969853
806	995126
811	996436
837	989262
901	988199
923	977066
934	990860
957	991171
989	975339
999	968823


# Graph based Music Recommender. Task 4
Build the edges of the type “artist-track”. Take the amount of times the track HAS BEEN listened by all users as the weight of the edge from the artist’s vertex to the track’s vertex.

Tip: group the dataframe by the columns “artistId” and “trackId” and use the function “count” of DF API. For each artist take top-100 tracks and normalize weights.

Sort the resulting Data Frame in descending order by the column norm_weight, and then in ascending order this time first by “id1”, then by “id2”. Take top 40 rows, select only the columns “id1”, “id2”, and print the columns “id1”, “id2” of the resulting dataframe.

The part of the result on the sample dataset:

<code>
...
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
... 
</code>

In [ ]:
def task4():
    X = data.select('artistId', 'trackId').groupBy('artistId', 'trackId').agg(f.count("*").alias('count')).cache()

    df = normalize(X, 'artistId', 'trackId', 'count', 1000) \
        .orderBy(f.desc('norm_count'), 'artistId', 'trackId') \
        .select('artistId', 'trackId') \
        .limit(40)

    for t1, t2 in df.collect():
        print("{}\t{}".format(t1,t2))

#task4()

967993	869415
967998	947428
968004	927380
968017	859321
968022	852786
968034	807671
968038	964150
968042	835935
968043	913568
968046	935077
968047	806127
968065	907906
968073	964586
968086	813446
968092	837129
968118	914441
968125	821410
968140	953008
968148	877445
968161	809793
968163	803065
968168	876119
968189	858639
968221	896937
968224	892880
968232	825536
968237	932845
968238	939177
968241	879045
968242	911250
968248	953554
968255	808494
968259	880230
968265	950148
968266	824437
968269	913243
968272	816049
968278	946743
968285	847460
968286	940006


# Graph based Music Recommender. Task 5
For the user with Id 776748 find all the tracks and artists connected to him. Use original dataframe not a normalized one. Sort founded items first by artist then by name in ascending order, leave only columns ”Artist” and “Name” and print top-40.

Each output line can take one of the following forms:

<code>
Artist: {artist-name} {track-name}
Artist: {artist-name} Artist: {artist-name}
</code>

These two forms help distinguish “user-track” suggestions (as shown in 1) from “user-artist” suggestions (as shown in 2).

The part of the result on the sample dataset:

<code>
...
Artist: Blur Artist: Blur
Artist: Blur Girls and Boys
Artist: Clawfinger Artist: Clawfinger
Artist: Clawfinger Nothing Going On
Artist: Disturbed Artist: Disturbed
...
</code>

In [ ]:
def task5():
    log = data.filter(f.col('userId') == 776748).cache()
    tracks = meta.filter(f.col('type') == u'track').withColumnRenamed('Id', 'trackId')
    artists = meta.filter(f.col('type') == u'artist').withColumnRenamed('Id', 'artistId')

    A = log.join(tracks, on=['trackId']).select('Artist', 'Name')
    B = log.join(artists, on=['artistId']).select('Artist', 'Name')

    df = A.union(B) \
        .orderBy('Artist', 'Name') \
        .select('Artist', 'Name') \
        .distinct() \
        .limit(40)

    for t1, t2 in df.collect():
        print("{}\t{}".format(t1,t2))
        
#task5()

Artist: 3 Doors Down	Artist: 3 Doors Down
Artist: 3 Doors Down	Kryptonite
Artist: 311	Artist: 311
Artist: 311	Beautiful disaster
Artist: Blur	Artist: Blur
Artist: Blur	Girls and Boys
Artist: Clawfinger	Artist: Clawfinger
Artist: Clawfinger	Nothing Going On
Artist: Disturbed	Artist: Disturbed
Artist: Disturbed	The Vengeful One
Artist: Gotthard	Artist: Gotthard
Artist: Gotthard	Eagle
Artist: Green Day	21 Guns
Artist: Green Day	Artist: Green Day
Artist: Green Day	Kill The DJ
Artist: Iggy Pop	Artist: Iggy Pop
Artist: Iggy Pop	Sunday
Artist: Korn	Artist: Korn
Artist: Korn	Here To Stay
Artist: Linkin Park	Artist: Linkin Park
Artist: Linkin Park	In The End
Artist: Linkin Park	Numb
Artist: Lordi	Artist: Lordi
Artist: Lordi	Hard Rock Hallelujah
Artist: Nickelback	Artist: Nickelback
Artist: Nickelback	She Keeps Me Up
Artist: Nomy	Artist: Nomy
Artist: Nomy	Cocaine
Artist: Papa Roach	Artist: Papa Roach
Artist: Papa Roach	Getting Away With Murder
Artist: Rise Against	Artist: Rise Against
Artist: Ri